In [1]:
%pip install rio-tiler -q
%pip install ipyleaflet -q
%pip install fastapi -q
%pip install uvicorn -q
%pip install morecantile -q
%pip install mercantile -q

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


I chose mercantile for getting the bounding box of the tile because:

- Specialized for Web Mercator Tiles: mercantile is specifically designed to work with the Slippy Map tiling scheme used in web mapping applications like OpenStreetMap and Mapbox. It provides utilities to convert between tile indices (x, y, z) and geographic coordinates (longitude, latitude).
- Lightweight and Efficient: It's a lightweight library that focuses solely on tile calculations, making it efficient for this specific task without adding unnecessary overhead.
- Ease of Use: It has straightforward functions like bounds(x, y, z) that directly give you the geographic bounding box of a tile, simplifying the code.

In [16]:
import os
import numpy as np
import threading
import asyncio

from io import BytesIO

import mercantile
import morecantile

from fastapi import FastAPI, HTTPException

from starlette.requests import Request
from starlette.responses import Response
from starlette.middleware.cors import CORSMiddleware
import uvicorn

from rio_tiler.io import Reader
from rio_tiler.profiles import img_profiles
from rio_tiler.errors import TileOutsideBounds
from rio_tiler.io import STACReader
from rasterio.enums import Resampling
from ipyleaflet import Map, TileLayer, basemaps, basemap_to_tiles, projections
from IPython.display import display
import logging
from functools import lru_cache

from random import randint

logging.basicConfig(level=logging.INFO)

In [3]:
#?Reader

In [4]:
#?STACReader

- `GDAL_CACHEMAX`: Increases the cache size for GDAL operations.
- `GDAL_NUM_THREADS`: Utilizes all available CPU cores.
- `CPL_VSIL_CURL_ALLOWED_EXTENSIONS`: Restricts allowed file extensions to reduce overhead.
- `VSI_CACHE` and `VSI_CACHE_SIZE`: Enable GDAL's built-in caching mechanism for remote files.

In [5]:
logging.basicConfig(level=logging.INFO)

# Define the Tile Matrix Set
tms = morecantile.tms.get("WebMercatorQuad")

# Set GDAL environment variables for performance
# Set GDAL environment variables for performance
os.environ["GDAL_CACHEMAX"] = "512"  # Cache size in MB
os.environ["GDAL_NUM_THREADS"] = "ALL_CPUS"
os.environ["CPL_VSIL_CURL_ALLOWED_EXTENSIONS"] = ".tif"
os.environ["VSI_CACHE"] = "TRUE"
os.environ["VSI_CACHE_SIZE"] = "536870912"  # 512 MB


In [6]:
cog_url='https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/54/T/WN/2023/8/S2A_54TWN_20230815_0_L2A/TCI.tif'

resampling_methods = [Resampling.nearest, Resampling.bilinear, Resampling.cubic]

In [7]:
app = FastAPI()

# Enable CORS
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],  # Adjust as needed
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

- `tilesize=256`: Standard tile size for web mapping.
- `resampling_method="bilinear"`: Smooth resampling method.


        - Options: 'nearest', 'bilinear', 'cubic', 'lanczos', etc.
        - Performance: 'nearest' is fastest but may produce blocky images. 'bilinear' offers a good balance.


- `unscale=True`: Applies scaling factors if the data is stored with offsets/scales.
- `use_overviews=True`: Uses pre-built overviews (pyramids) in the COG for faster access at lower zoom levels.
- `nodata=0`: Defines nodata value to handle transparency.
- `Image Rendering`: Uses DEFLATE compression to reduce file size

In [17]:
def generate_tile(x:int, y:int, z:int, cog_url, tms, resampling_method=Resampling.bilinear):
    """
    Generate a tile from a COG at the specified x, y, z with optimized settings.
    """
    try:
        with Reader(cog_url) as cog: #using context manager is a good way to make sure the dataset are closed when we exit.
            img = cog.tile(
                x, y, z,
                tilesize=256,
                tile_matrix_set=tms,
                resampling_method=resampling_method,  # Faster resampling
                use_overviews=True,  # Utilize overviews for lower zoom levels
                unscale=True,  # Apply scaling if necessary
                nodata=0  # Handle nodata values,
            )
        # Optimize image rendering
        png_profile = img_profiles.get("png")
        png_profile.update({"compress": "DEFLATE"})
        content =  img.render(img_format="PNG", **png_profile)
        return content
    except TileOutsideBounds:
        print(f"TileOutsideBounds: z={z}, x={x}, y={y}")
        return None
    except Exception as e:
        print(f"Error generating tile z={z}, x={x}, y={y}: {e}")
        return None


In [18]:
def simulate_tile_requests(cog_url, num_tiles=100):
    """
    Simulate multiple tile requests for benchmarking. This will NOT produce a working functional tile set as it picks randome tiles from a given zoom level.
    """

    # Define zoom levels and tile ranges (adjust based on your dataset)
    zoom_levels = [10, 12, 14, 16]
    tile_results = []

    for _ in range(num_tiles):
        z = np.random.choice(zoom_levels)
        x = randint(0, 2 ** z - 1)
        y = randint(0, 2 ** z - 1)
        tile_data = generate_tile(x, y, z, cog_url)
        tile_results.append(tile_data)
    
    return tile_results

In [23]:
@app.get("/")
async def root():
    return {"message": "Tile server is running."}

@app.get(
        r"/{z}/{x}/{y}.png",
        responses={
            200:{
                "content":{"image/png":{}},"description":"return an image"
            }
        },
        response_class=Response,
        description="read COG and return tile",
        )
async def tile_route(z: int, x: int, y: int):
    print(f"Received tile request: z={z}, x={x}, y={y}")
    y_tms = (2 ** z - 1) - y
    tile_data = generate_tile(x, y_tms, z, cog_url)
    #tile_data = generate_tile(x, y, z, cog_url, tms=tms)
    if tile_data:
        return Response(content=tile_data, media_type="image/png")
    else:
        print(f"No tile data for z={z}, x={x}, y={y}")
        raise HTTPException(status_code=204)

In [11]:
def run_app():
    uvicorn.run(app, host='0.0.0.0', port=5000, log_level="debug", access_log=True)

In [24]:
# Start the FastAPI app in a separate thread
thread = threading.Thread(target=run_app, daemon=True)
thread.start()

INFO:     Started server process [30990]
INFO:     Waiting for application startup.


INFO:     Application startup complete.
ERROR:    [Errno 98] error while attempting to bind on address ('0.0.0.0', 5000): address already in use
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.


In [ ]:
# %%time
# test_tile = generate_tile(0, 0, 2, cog_url, tms)

INFO:     Started server process [30990]


In [20]:
with Reader(cog_url) as cog: #temporary hacky way to get the extent for the map viz below.
    info = cog.info()

bbox = info.bounds
center_lon = (bbox[0] + bbox[2]) / 2
center_lat = (bbox[1] + bbox[3]) / 2
center = (center_lat, center_lon)

print(center)

INFO:botocore.credentials:Found credentials in environment variables.
INFO:botocore.credentials:Found credentials in environment variables.


(42.85458889075426, 141.67710026972452)


In [21]:
z = 12
tile = mercantile.tile(center_lon, center_lat, z)
x = tile.x
y = tile.y
print(f"Tile coordinates: z={z}, x={x}, y={y}")

Tile coordinates: z=12, x=3659, y=1507


In [25]:
# Create the map
m = Map(center=center, zoom=info.minzoom)

# Define the tile layer
tile_layer = TileLayer(
    url='http://localhost:5000/tiles/{z}/{x}/{y}.png',
    attribution='Dynamic Tiles via rio-tiler',
    name='Dynamic Tiles',
    max_zoom=info.maxzoom,
    min_zoom=info.minzoom
)

# Add the tile layer to the map
m.add_layer(tile_layer)

# Display the map
display(m)


Map(center=[42.85458889075426, 141.67710026972452], controls=(ZoomControl(options=['position', 'zoom_in_text',…

In [16]:
# %%time
# # Simulate 100 tile requests
# tile_results = simulate_tile_requests(cog_url, num_tiles=100)


Benchmark with Different Resampling Methods:

In [17]:
# for method in resampling_methods:
#     print(f"Testing resampling method: {method.name}")
#     def generate_tile(x, y, z, cog_url):
#         return generate_tile_resampling(x, y, z, cog_url, resampling_method=method)
    
#     %%time
#     tile_results = simulate_tile_requests(cog_url, num_tiles=100)